In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from channel.ion_analysis import ion_coordination
from plot.plot_utilities import edgeformat, shared_xylabels, savefig
from database.query import get_trajid

In [ ]:
%store -r traj_ids
%store -r transloc_df

In [ ]:
cla_coord = ion_coordination()
cla_coord.calc_polar()
cla_coord.calc_nonpolar()

In [ ]:
zmin = 90
zmax = 150
bin_width = 1

cla_coord.prepare()

In [ ]:
low_voltage_trajids = get_trajid(voltage=(-0.01, 0.01))
low_voltage_trajids = np.intersect1d(low_voltage_trajids, traj_ids)

# transloc_onpath = transloc_df.query("(path_assign == 2 | path_assign == 0) & traj_id in @low_voltage_trajids")['transloc_id']
transloc_onpath = transloc_df.query("path_assign == 1 & traj_id in @low_voltage_trajids")['transloc_id']
# transloc_onpath = transloc_df.query("traj_id in @low_voltage_trajids")['transloc_id']

cla_coord.analyze('traj_id in @low_voltage_trajids & transloc_id in @transloc_onpath', low_voltage_trajids=low_voltage_trajids, transloc_onpath=list(transloc_onpath))

# Analysis

In [ ]:
protein_resnames = ["ASP", "GLU", "LYS", "ARG", "HIS", "HSD", "TYR", "TRP", "PHE", "ASN", "GLN", "SER", "THR", "MET", "ALA", "VAL", "LEU", "ILE", "PRO", "GLY", "CYS"]
other_resnames = ["POPC", "TIP3", "SOD", "CLA"]

In [ ]:
plot_zmin = 90
plot_zmax = 145

In [ ]:
def plot_vs_z(func):
    def _plot(axs, horizontal=True, fill_between=False, **plot_kwargs):
        resbind = func(cla_coord)
        if horizontal:
            axs.plot(cla_coord.bin_z, resbind, **plot_kwargs)
            if fill_between:
                axs.fill_between(cla_coord.bin_z, np.zeros(len(cla_coord.bin_z)), resbind, color=plot_kwargs['color'], alpha=0.5)
        else:
            axs.plot(resbind, cla_coord.bin_z, **plot_kwargs)
            if fill_between:
                axs.fill_betweenx(cla_coord.bin_z, np.zeros(len(cla_coord.bin_z)), resbind, color=plot_kwargs['color'], alpha=0.5)
        return resbind
    return _plot

@plot_vs_z
def all_coord_contribute(cla_coord):
    # All
    all_resbind = np.sum(list(cla_coord.resid_bind.values()), axis=0)
    return all_resbind

@plot_vs_z
def water_coord_contribute(cla_coord):
    # Water
    water_resbind = cla_coord.resid_bind['TIP3']
    return water_resbind

@plot_vs_z
def protein_coord_contribute(cla_coord):
    # Protein
    # TODO: more rigorous check
    prot_resbind = [vals for res, vals in cla_coord.resid_bind.items() if res not in other_resnames]
    prot_resbind = np.sum(prot_resbind, axis=0)
    return prot_resbind

@plot_vs_z
def nonpolar_coord_contribute(cla_coord):
    # Non-polar protein residues
    prot_nonpolar_resbind = [vals for res, vals in cla_coord.resid_bind_nonpolar.items() if res not in other_resnames]
    # prot_nonpolar_resbind = np.sum(np.nan_to_num(prot_nonpolar_resbind), axis=0)
    prot_nonpolar_resbind = np.sum(prot_nonpolar_resbind, axis=0)
    return prot_nonpolar_resbind

In [ ]:
resids_of_interest = ["K190", "R248", "R303", "R352", "W356", "R1097", "K95", "R134", "S1141", "S341", "N1138", "T338", "R334", "F337", "Y917", "Y914", "S1118", "R117", "R104", "POPC"]

# POPC is excluded: beyond the scope of this work and is outside of the pore
# R104 is excluded: it does show a peak in the n_solv vs z plot in the grey region
# However, it is not actually inside the bottleneck region (shown as grey) or contribute to the selectivity filter
# This is because 1-6 pathway runs sideways and not straight up along z anymore at this region
# In short, the structure of the permeation pathway in this region is peculiar
exclude_residues = ["POPC", "R104"]

In [ ]:
# annotation_position_shift = {"Y914": (-0.1,2.5), "Y917": (0,-1), "R117": (-0.2,-1), "F337": (0,-1), "S341": (-0.1,-1), "R352": (0,1), "R1097": (0,1), "K190": (-0.1,-1.5)}
annotation_position_shift = {}
manual_color = {}

fig, Axs = plt.subplots(1, 2, figsize=(8,10), sharey=True)
for axs in Axs:
    edgeformat(axs)
    # set tick font size to 14
    axs.tick_params(labelsize=14)

shared_xylabels(fig, ylabel=r"$z$ [$\mathrm{\AA}$]", xlabel=r"$n_{solv}$(Cl-)", fontsize=20)

##### Residue specific #####
for res in resids_of_interest:

    # if res in exclude_residues:
    #     continue

    resid_bind = cla_coord.resid_bind[res]
    resid_bind_nonpolar = cla_coord.resid_bind_nonpolar[res]
    # Show the non-polar residues in resids_of_interest even if they have no major contribution
    if max(resid_bind) < 0.1:
        if max(resid_bind_nonpolar) > 0:
            resid_bind = resid_bind_nonpolar
        else:
            continue
    
    if manual_color.get(res) is not None:
        Axs[1].plot(resid_bind, cla_coord.bin_z, color=manual_color[res], lw=2)
    else:
        Axs[1].plot(resid_bind, cla_coord.bin_z, lw=2)

    # Get the color of the line
    color = Axs[1].get_lines()[-1].get_color()

    text_xshift, text_yshift = annotation_position_shift.get(res, (0,0))
    Axs[1].annotate(res, 
                    xy=(np.nanmax(resid_bind), cla_coord.bin_z[np.nanargmax(resid_bind)]), 
                    xytext=(np.nanmax(resid_bind)+text_xshift+0.01, cla_coord.bin_z[np.nanargmax(resid_bind)]+text_yshift-1), 
                    fontsize=16, color=color, weight='bold')

Axs[1].set_ylim(plot_zmin,plot_zmax)
Axs[1].set_yticks(np.arange(plot_zmin,plot_zmax+5,5))
Axs[1].set_xlim(0,1)
Axs[1].grid(True, ls='--')

##### All, Water, Protein #####
# All
all_resbind = all_coord_contribute(Axs[0], horizontal=False, label="all", color='green', lw=2)
# Water
water_resbind = water_coord_contribute(Axs[0], horizontal=False, label="water", color='red', lw=2)
# Protein
prot_resbind = protein_coord_contribute(Axs[0], horizontal=False, label="protein", color='black', lw=2)
# Non-polar protein residues
prot_nonpolar_resbind = nonpolar_coord_contribute(Axs[0], horizontal=False, label="non-polar protein", color='purple', lw=2)

for axs in Axs:
    axs.fill_between([0,8], 125, 135, color='gray', alpha=0.2)

Axs[0].set_xlim(0,8)
Axs[0].set_xticks(np.arange(0,8+1))
Axs[0].set_ylim(plot_zmin,plot_zmax)
# Axs[1].legend()
Axs[0].grid(True, ls='--')
Axs[0].set_yticks(np.arange(plot_zmin,plot_zmax+5,5))
Axs[0].set_yticklabels(np.arange(plot_zmin,plot_zmax+5,5)-130)

# savefig("central_pathway_ionenv_lowU_v.pdf")

In [ ]:
#annotation_position_shift = {"Y914": (-0.1,2.5), "Y917": (0,-1), "R117": (-0.2,-1), "F337": (0,-1), "S341": (-0.1,-1), "R352": (0,1), "R1097": (0,1), "K190": (-0.1,-1.5)}
annotation_position_shift = {}
manual_color = {}

fig, Axs = plt.subplots(1, 2, figsize=(8,8), sharey=True)
for axs in Axs:
    edgeformat(axs)
    # set tick font size to 14
    axs.tick_params(labelsize=14)

shared_xylabels(fig, ylabel=r"$z$ [$\mathrm{\AA}$]", xlabel=r"$n_{solv}$(Cl-)", fontsize=20)

##### Residue specific #####
for res in resids_of_interest:

    if res in exclude_residues:
        continue

    resid_bind = cla_coord.resid_bind[res]
    resid_bind_nonpolar = cla_coord.resid_bind_nonpolar[res]
    if max(resid_bind) < 0.1:
        if max(resid_bind_nonpolar) > 0:
            resid_bind = resid_bind_nonpolar
        else:
            continue
    
    if manual_color.get(res) is not None:
        Axs[1].plot(resid_bind, cla_coord.bin_z, color=manual_color[res], lw=2)
    else:
        Axs[1].plot(resid_bind, cla_coord.bin_z, lw=2)

    # Get the color of the line
    color = Axs[1].get_lines()[-1].get_color()

    text_xshift, text_yshift = annotation_position_shift.get(res, (0,0))
    Axs[1].annotate(res, 
                    xy=(np.nanmax(resid_bind), cla_coord.bin_z[np.nanargmax(resid_bind)]), 
                    xytext=(np.nanmax(resid_bind)+text_xshift+0.01, cla_coord.bin_z[np.nanargmax(resid_bind)]+text_yshift-1), 
                    fontsize=16, color=color, weight='bold')

Axs[1].set_ylim(plot_zmin,plot_zmax)
Axs[1].set_yticks(np.arange(plot_zmin,plot_zmax+5,5))
Axs[1].set_xlim(0,1)
Axs[1].grid(True, ls='--')

##### All, Water, Protein #####
# All
all_resbind = all_coord_contribute(Axs[0], horizontal=False, label="all", color='green', lw=2)
# Water
water_resbind = water_coord_contribute(Axs[0], horizontal=False, label="water", color='red', lw=2)
# Protein
prot_resbind = protein_coord_contribute(Axs[0], horizontal=False, label="protein", color='black', lw=2)
# Non-polar protein residues
prot_nonpolar_resbind = nonpolar_coord_contribute(Axs[0], horizontal=False, label="non-polar protein", color='purple', lw=2)

for axs in Axs:
    axs.fill_between([0,8], 125, 135, color='gray', alpha=0.2)

Axs[0].set_xlim(0,8)
Axs[0].set_xticks(np.arange(0,8+1))
Axs[0].set_ylim(plot_zmin+30,plot_zmax)
# Axs[1].legend()
Axs[0].grid(True, ls='--')
Axs[0].set_yticks(np.arange(plot_zmin+30,plot_zmax+5,5))
Axs[0].set_yticklabels(np.arange(plot_zmin+30,plot_zmax+5,5)-130)

# savefig("central_pathway_ionenv_lowU_vshort.pdf")

# Water histogram

In [ ]:
n_water_bound = cla_coord.polar_df.query('z > 160 | z < 10')['TIP3']
n_water_bound.value_counts()

fig, axs = plt.subplots()
edgeformat(axs)

axs.bar(n_water_bound.value_counts().index, n_water_bound.value_counts().values / len(n_water_bound), color='green')
axs.set_xlim(0,10)
axs.set_ylim(0,1)
axs.set_xlabel(r"$N_{\mathrm{{H_2}O}}$ in 1st coord. shell", fontsize=16)
axs.set_ylabel("Probability", fontsize=16)

# savefig("coord_shell_water-central.pdf")

In [ ]:
print(n_water_bound.mean(), n_water_bound.std())

In [ ]:
fig, axs = plt.subplots()
edgeformat(axs)


n_water_bound = cla_coord.polar_df.query('z > 160 | z < 10')['TIP3']
n_water_bound.value_counts()
axs.bar(n_water_bound.value_counts().index, n_water_bound.value_counts().values / len(n_water_bound), width=0.4, align='edge', color='grey', zorder=2, alpha=0.5)

transloc_central_path = transloc_df.query("path_assign == 2 | path_assign == 0")['transloc_id']
n_water_bound = cla_coord.polar_df.query('z > 125 & z < 135 & transloc_id in @transloc_central_path')['TIP3']
n_water_bound.value_counts()
axs.bar(n_water_bound.value_counts().index, n_water_bound.value_counts().values / len(n_water_bound), width=-0.4, align='edge', color='green', zorder=2)

axs.set_xlim(0,10)
axs.set_ylim(0,0.5)
axs.set_xlabel(r"$N_{\mathrm{{H_2}O}} \mathrm{(coord)}$", fontsize=16)
axs.set_ylabel("Probability", fontsize=16)
axs.grid(True, ls='--', zorder=1)
axs.set_xticks(np.arange(-1,9)+1)
axs.set_xlim(-1,10)

# savefig("coord_shell_water-central.pdf")

In [ ]:
print(n_water_bound.mean(), n_water_bound.std())

In [ ]:
fig, axs = plt.subplots()
edgeformat(axs)

n_water_bound = cla_coord.polar_df.query('z > 160 | z < 10')['TIP3']
n_water_bound.value_counts()
axs.bar(n_water_bound.value_counts().index, n_water_bound.value_counts().values / len(n_water_bound), width=0.4, align='edge', color='grey', zorder=2, alpha=0.5)

transloc_side_path = transloc_df.query("path_assign == 1")['transloc_id']
n_water_bound = cla_coord.polar_df.query('z > 125 & z < 135 & transloc_id in @transloc_side_path')['TIP3']
n_water_bound.value_counts()
axs.bar(n_water_bound.value_counts().index, n_water_bound.value_counts().values / len(n_water_bound), width=-0.4, align='edge', color='blue', zorder=2)

axs.set_xlim(0,10)
axs.set_ylim(0,0.5)
axs.set_xlabel(r"$N_{\mathrm{{H_2}O}} \mathrm{(coord)}$", fontsize=16)
axs.set_ylabel("Probability", fontsize=16)
axs.grid(True, ls='--', zorder=1)
axs.set_xticks(np.arange(-1,9)+1)
axs.set_xlim(-1,10)

# savefig("coord_shell_water-1-6.pdf")

In [ ]:
print(n_water_bound.mean(), n_water_bound.std())